In [68]:
from options.test_options import TestOptions
from data import DataLoader
from models import create_model
from util.writer import Writer
import numpy as np
import random
import torch.nn as nn
import torch

In [69]:
def test_attacked_model(model, dataset, writer):
    writer.reset_counter()
    attacked_model_outputs = []
    
    for i, data in enumerate(dataset):
        model.set_input(data)
        output, ncorrect, nexamples = model.test() 
        attacked_model_outputs.append(output)
        writer.update_counter(ncorrect, nexamples)
        
    writer.print_acc(-1, writer.acc)
    return attacked_model_outputs 
    
def find_new_vertex_index(vertices_edges, edge_index, old_vertex_index):
    for new_vertex_index, new_vertex_edges in enumerate(vertices_edges):
            for new_edge_index in new_vertex_edges:
                if(new_edge_index == edge_index and new_vertex_index != old_vertex_index):
                    return new_vertex_index
                
def get_random_walk(mesh, random_walk_size):
    walk_steps = 0
    random_walk_vertices = []
    random_walk_indices = []
    vertex_index = random.randint(0, len(mesh.vs)-1)
    
    while walk_steps < random_walk_size: 
        random_walk_vertices.append(mesh.vs[vertex_index])
        random_walk_indices.append(vertex_index)        
        walk_steps += 1  
        
        vertex_edges = mesh.ve[vertex_index]
        random_edge_index = random.randint(0, len(vertex_edges)-1)  
        new_edge_index = vertex_edges[random_edge_index]
        vertex_index = find_new_vertex_index(mesh.ve, new_edge_index, random_walk_indices[-1])
        
        #Prevents random walk from crossing over itself
        count_of_vertex_edge_attempts = 0
        walk_steps_backwards = 0
        while(vertex_index in random_walk_indices):
            if(count_of_vertex_edge_attempts >= len(vertex_edges)-1):
                walk_steps_backwards += 1
                go_back_to_index = walk_steps-walk_steps_backwards
                
                if(go_back_to_index == 0):
                    # Trying again, mesh seems to be broken?
                    return get_random_walk(mesh, random_walk_size)
                
                vertex_edges = mesh.ve[random_walk_indices[go_back_to_index]]
                count_of_vertex_edge_attempts = 0
                
            random_edge_index = (random_edge_index + 1) % len(vertex_edges)
            new_edge_index = vertex_edges[random_edge_index]
            vertex_index = find_new_vertex_index(mesh.ve, new_edge_index, random_walk_indices[-1])
            count_of_vertex_edge_attempts += 1        
    
    return torch.FloatTensor(random_walk_vertices), random_walk_indices

In [121]:
def train_imitating_network(imitating_nn, criterion, optimizer, random_walk_vertices, attacked_nn_output):

    output = imitating_nn(random_walk_vertices)
    loss = criterion(output, attacked_nn_output)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step() 

    return output, loss.item()

In [122]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        
        combined = torch.cat((input, hidden), -1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

class Imitating_NN(nn.Module):
    def __init__(self, input_size, output_size):
        super(Imitating_NN, self).__init__()
        
        self.num_vertices = 3
        self.scaling_factor = 10
        
        self.first_linear = nn.Linear(input_size, input_size*self.scaling_factor)
        self.second_linear = nn.Linear(input_size*self.scaling_factor, input_size*self.scaling_factor)
        self.rnn = RNN(self.scaling_factor*input_size, input_size, self.scaling_factor*input_size)
        self.third_linear = nn.Linear(self.scaling_factor*input_size, output_size)
        
    def forward(self, random_walk_vertices):
        
        transformed_random_walk = self.first_linear(random_walk_vertices)
        transformed_random_walk = self.second_linear(transformed_random_walk)
        
        hidden = self.rnn.initHidden()
        for step in transformed_random_walk:
            output, hidden = self.rnn(torch.reshape(step, (1, self.num_vertices*self.scaling_factor)), hidden)
            
        output = self.second_linear(output)
            
        return output
        

# Orchestration

In [123]:
#TODO: test accuracy impact from random movement of vertices

testing_opt = TestOptions().parse()

testing_opt.serial_batches = True
dataloader = DataLoader(testing_opt)
mesh_cnn = create_model(testing_opt)
opt_writer = Writer(testing_opt)

random_walk_size = 50
num_categories = 30
num_vertice_coordinates = 3
imitating_nn = Imitating_NN(num_vertice_coordinates, num_categories)
criterion = nn.KLDivLoss(reduction="batchmean")
optimizer = torch.optim.Adam(imitating_nn.parameters(), lr = 0.0001)

loaded mean / std from cache
loading the model from ./checkpoints/shrec16/latest_net.pth


In [119]:
# Record testing accuracy before attack
attacked_model_outputs = test_attacked_model(mesh_cnn, dataloader, opt_writer)

epoch: -1, TEST ACC: [99.167 %]



In [124]:
# Stage attack
for epoch in range(10):
    epoch_loss = 0
    
    for i, data in enumerate(dataloader):
        print("i: " + str(i))
        print("Mesh labels: " + str(data["label"]))
    
        mesh = data["mesh"][0]
        random_walk_vertices, random_walk_indices = get_random_walk(mesh, random_walk_size)
    
        imitating_nn_output, loss = train_imitating_network(imitating_nn, criterion, optimizer, random_walk_vertices, attacked_model_outputs[i])
        epoch_loss += loss
    
        print(attacked_model_outputs[i])
        print(imitating_nn_output)
    
    print("epoch: " + str(epoch) + ", loss: " + str(epoch_loss))
    

i: 0
Mesh labels: [0]
tensor([[  5.4885, -19.8533, -21.4682,  -9.6839,  -8.4958,  -3.6247,  -4.4009,
          -6.7783, -23.9704, -19.1216, -15.8959,  -6.0566,  -4.8584, -11.2263,
         -16.3660, -13.7719,  -8.6655, -10.7375,  -5.2502, -18.5240,  -0.8722,
         -19.7822, -22.9073,  -3.3643, -25.6530,  -7.8275, -25.9369, -18.4495,
          -3.2837, -13.6820]])
tensor([[-0.1210, -0.0674,  0.0266, -0.0531, -0.1874, -0.0510,  0.0306,  0.1524,
         -0.0869,  0.1351,  0.0221,  0.0087, -0.0008, -0.0260,  0.1878, -0.1341,
         -0.0091,  0.1530,  0.3260, -0.2276, -0.0435, -0.1196,  0.3247,  0.0136,
         -0.0144, -0.0782,  0.2042, -0.1304, -0.0685, -0.0576]],
       grad_fn=<AddmmBackward0>)
i: 1
Mesh labels: [0]
tensor([[  6.3456, -20.1829, -23.7263, -11.1970, -10.4479,  -3.5765,  -4.1582,
          -5.3284, -26.1639, -20.5226, -15.8104,  -4.7861,  -4.6728, -11.7994,
         -17.7693, -14.5876,  -7.8231, -13.3061,  -7.3532, -18.4909,  -1.7529,
         -21.3954, -25.2309,  -

tensor([[-11.5600, -12.2807,  -8.4907,   4.3490,  -3.9629,  -6.6956,  -5.1210,
         -17.6667, -16.9735,  -6.5677, -17.3407, -22.6724, -22.5112, -12.0273,
          -9.3018,  -2.4722, -22.2240,  -8.2088,   0.6924, -21.5803, -12.2139,
          -6.7683, -23.1302,  -5.0496, -22.1565, -13.7152, -21.1941, -12.2661,
         -10.6854, -24.5271]])
tensor([[-0.0807, -0.0968,  0.0579, -0.0031, -0.2299, -0.0425,  0.0369,  0.1746,
         -0.0537,  0.0506,  0.1215, -0.0639,  0.0195,  0.0141,  0.1374, -0.1052,
          0.0374,  0.1472,  0.3343, -0.2354, -0.0790, -0.0903,  0.3866,  0.0048,
         -0.0126, -0.1334,  0.2438, -0.0851, -0.0726, -0.0877]],
       grad_fn=<AddmmBackward0>)
i: 13
Mesh labels: [3]
tensor([[-12.6880, -12.8750, -10.0355,   5.0915,  -2.9794,  -7.6716,  -4.5700,
         -18.3993, -15.3633,  -5.6254, -17.1177, -22.0048, -21.5706, -11.0255,
          -8.2964,  -3.0174, -23.5324, -10.5834,  -1.8890, -19.7286, -11.2272,
          -7.6164, -20.7858,  -7.4569, -21.0455, -11

tensor([[-14.5027, -13.2319, -21.4615, -12.8570, -17.1793, -11.6074,  -2.7180,
         -12.7763, -23.4383, -11.8774, -11.1201, -11.7537, -21.2265, -19.1802,
         -10.7729,  -9.0745, -16.8068, -27.7212, -18.5842, -12.0273, -22.8469,
         -23.7130, -26.2228, -11.3570, -20.5803, -15.3374, -26.3384, -21.8032,
         -11.9231, -15.5972]])
tensor([[-0.0823, -0.0682,  0.0615, -0.0186, -0.1913, -0.0440,  0.0316,  0.1631,
         -0.0832,  0.0981,  0.0652, -0.0247,  0.0170, -0.0045,  0.1654, -0.1319,
          0.0034,  0.1495,  0.3352, -0.2402, -0.0577, -0.1049,  0.3432,  0.0088,
          0.0013, -0.1058,  0.2326, -0.1185, -0.0671, -0.0776]],
       grad_fn=<AddmmBackward0>)
i: 26
Mesh labels: [6]
tensor([[-14.8296, -18.9902, -28.8402, -15.5756, -15.2484, -11.4294,  -2.6644,
         -13.2581, -24.5787, -13.1838, -15.5056,  -8.6107, -15.0708, -15.5415,
         -12.3020, -13.9708, -18.9744, -28.3200, -21.5157,  -9.3443, -19.8563,
         -25.5049, -21.4479, -16.4845, -22.2518, -11

tensor([[-26.9754,  -9.1055, -11.6346,  -6.0669, -11.5090, -14.4946,  -6.7320,
         -20.5614,  -9.8262,   2.1387, -16.3481, -24.1510, -28.9189, -11.5371,
          -5.3347,  -2.8517, -28.9497, -20.0269, -15.0855,  -8.7562, -25.8955,
         -10.0486, -12.0381, -17.0934, -13.8759, -14.2032,  -9.0038, -14.6300,
         -19.3966, -19.1076]])
tensor([[-0.0861, -0.0670,  0.0524,  0.0148, -0.1788, -0.0490,  0.0312,  0.1377,
         -0.1061,  0.0919,  0.0408, -0.0255, -0.0352, -0.0020,  0.1687, -0.1534,
         -0.0083,  0.1601,  0.3382, -0.2432, -0.0598, -0.0959,  0.3168, -0.0225,
          0.0276, -0.1065,  0.2369, -0.1137, -0.0348, -0.0877]],
       grad_fn=<AddmmBackward0>)
i: 39
Mesh labels: [9]
tensor([[-28.4242,  -8.2474, -13.6672,  -8.3598, -13.4178, -15.0705,  -6.6848,
         -19.5868,  -9.2745,   2.4944, -16.3123, -23.3888, -29.2222, -11.4324,
          -6.3626,  -4.3604, -29.0648, -22.5601, -18.4032,  -6.6493, -27.5186,
         -11.1720, -10.5156, -19.5232, -12.9198, -14

tensor([[-12.7686, -17.5714, -30.0671, -10.8246,  -6.6834,  -4.9487,  -5.6762,
          -8.2139, -19.3342,  -7.7499, -27.4529, -14.2749,  -6.1452,   5.8931,
         -16.9944, -13.9010, -22.8718, -15.9904, -17.8295,  -8.2325,  -8.0531,
          -7.1127, -12.5369, -20.3526, -26.0155,  -5.7977,  -4.3818,  -9.9493,
         -18.2877, -12.6711]])
tensor([[-0.0680, -0.0653,  0.0750, -0.0305, -0.1958, -0.0344,  0.0425,  0.2021,
         -0.0407,  0.1004,  0.1047, -0.0084,  0.0566,  0.0180,  0.1627, -0.1252,
          0.0203,  0.1510,  0.3335, -0.2337, -0.0650, -0.0924,  0.3580,  0.0233,
          0.0004, -0.1230,  0.2383, -0.1192, -0.0737, -0.0588]],
       grad_fn=<AddmmBackward0>)
i: 54
Mesh labels: [13]
tensor([[-13.8156, -12.8400, -23.8924,  -9.2676,  -7.2072,  -5.5136,  -4.7157,
          -7.4916, -14.0714,  -3.9144, -22.4206, -13.1019,  -8.7493,   5.0935,
         -12.8874, -10.2377, -20.8854, -15.0960, -16.5194,  -4.8228, -10.2136,
          -5.1903,  -9.4925, -18.2005, -20.1399,  -

...

tensor([[-20.5121, -10.8599,  -7.5975, -11.9631, -13.1542, -18.6488,  -6.4280,
         -20.0937,  -4.2401,  -6.0960,  -2.7362, -11.5782, -19.2985, -22.1444,
           4.0644,  -8.5918, -17.2663, -21.8911, -12.9144,  -3.9298, -19.6721,
         -27.1996,  -7.6258, -14.0386,   0.8437,  -8.9811, -12.5892, -27.7664,
          -3.5568,  -2.5695]])
tensor([[ 0.4469,  1.1848,  1.0964,  0.0398, -0.0560,  1.0621, -0.2128,  1.2777,
          0.8472,  1.0485,  1.4631,  1.4952,  1.0593,  1.3540,  1.1611,  0.1123,
          1.0517,  0.7642,  1.0611, -0.4105,  0.5172,  0.7190,  0.8274,  0.8459,
          1.1529,  0.3429,  0.9726,  0.8497,  0.9649,  1.1967]],
       grad_fn=<AddmmBackward0>)
i: 60
Mesh labels: [15]
tensor([[-13.5967,  -2.3520,  -6.0668,  -3.6582, -14.0064,  -8.8274,  -3.0108,
          -9.4713, -15.9220,  -2.6821,  -7.0240, -15.6165, -25.0987, -13.4810,
          -5.9365,   4.0143, -15.2033, -18.3239,  -9.3359, -11.3389, -23.5738,
          -8.4257, -23.9934,  -2.9656, -15.4251, -1

tensor([[ -6.4295, -14.6801,  -4.1200,  -0.1680,  -4.3292,  -6.5845,  -6.2795,
         -17.6786, -15.8085, -10.2893, -15.0795, -19.4315, -19.0610, -15.7840,
          -7.7838,  -4.5055, -16.7387,  -1.5443,   8.1127, -22.9785,  -9.2824,
         -11.0889, -21.8150,   0.4739, -19.2742, -12.8191, -21.3896, -15.9492,
          -3.1681, -20.7893]])
tensor([[ 0.4845,  1.2184,  1.1023,  0.0570, -0.0554,  1.0443, -0.2338,  1.2779,
          0.8276,  1.0485,  1.4326,  1.5231,  1.0349,  1.4156,  1.2169,  0.1007,
          1.0853,  0.8374,  1.0945, -0.4705,  0.5388,  0.7196,  0.7947,  0.8473,
          1.1980,  0.3215,  1.0170,  0.8285,  1.0268,  1.1782]],
       grad_fn=<AddmmBackward0>)
i: 74
Mesh labels: [18]
tensor([[ -6.8667, -15.2529,  -4.2042,   0.6357,  -4.3796,  -7.5659,  -6.3964,
         -18.6289, -16.0172, -10.4212, -15.2902, -20.4248, -20.2556, -16.4315,
          -7.5203,  -4.4047, -18.0207,  -2.8373,   7.9993, -23.6170,  -9.4781,
         -11.8062, -22.7356,   0.1718, -19.7938, -1

tensor([[-17.1850,  -7.1073, -14.7938,  -1.7395,  -2.3789,   1.4221,  -6.6295,
         -13.9747, -20.0892,  -2.9650, -25.4700, -23.6302, -17.1402,  -1.8111,
         -17.4176,  -3.7117, -21.3280,  -0.1791,  -7.0395, -18.3350, -16.8462,
           9.6003, -18.4236, -11.0242, -24.9167, -19.4055, -13.2250,  -1.3964,
         -19.1988, -27.6331]])
tensor([[ 0.5098,  1.3045,  1.2048,  0.0881, -0.0967,  1.2593, -0.2383,  1.3944,
          0.9812,  1.0767,  1.6902,  1.6473,  1.1666,  1.5645,  1.2944,  0.2425,
          1.2816,  0.9328,  1.2223, -0.4549,  0.5852,  0.8309,  0.9160,  0.9368,
          1.2733,  0.3398,  1.0393,  1.0215,  1.0931,  1.3182]],
       grad_fn=<AddmmBackward0>)
i: 88
Mesh labels: [22]
tensor([[-24.1869, -16.2579, -18.5360, -14.4957,  -6.7891, -15.6602,  -7.5203,
         -21.3831,  -1.1955,  -1.4690, -14.5474, -12.5893, -12.0231,  -8.8962,
          -3.5067, -15.8422, -23.6550, -18.1094, -19.6146,   0.5762, -14.0230,
         -19.8573,   9.2143, -25.6817,  -3.6654,  -

tensor([[ -7.4048, -22.1733, -15.2929,  -6.4625,  -5.3935, -12.9865,  -5.0201,
         -18.1305, -11.8294, -10.7239, -16.0824, -12.8918, -12.5425, -11.4209,
          -7.5349, -11.3807, -20.0058, -15.6435,  -5.4711, -14.6736,  -5.6768,
         -20.9887, -15.0176,  -9.4945, -18.9987,  -0.6859, -14.4005, -25.4367,
          -7.3166, -11.3289]])
tensor([[ 0.5136,  1.3622,  1.2430,  0.0311, -0.1121,  1.3427, -0.2451,  1.4771,
          1.0490,  1.1936,  1.7429,  1.7623,  1.2658,  1.5850,  1.3845,  0.2761,
          1.3520,  0.9726,  1.2844, -0.4271,  0.6830,  0.9029,  0.9976,  1.0687,
          1.2975,  0.4189,  1.0361,  1.0119,  1.0980,  1.3878]],
       grad_fn=<AddmmBackward0>)
i: 101
Mesh labels: [25]
tensor([[-12.8098, -23.2901, -20.9864,  -9.1904,  -3.2161, -15.2389,  -4.7031,
         -18.8213,  -5.7028,  -6.9229, -17.6228, -10.7221,  -7.1677,  -4.3441,
          -5.7731, -15.3477, -23.2659, -19.0057, -12.8664,  -6.1625,  -3.2857,
         -20.8899,  -3.9397, -19.1224, -13.9923,  

tensor([[ -5.6747, -16.5607, -13.8552, -12.4289,  -9.0309,  -9.1177,  -7.3269,
         -13.4645, -19.0084, -16.3229,  -9.7802,  -8.4972,  -6.0034, -17.5821,
          -5.2400, -11.8945,  -9.7701, -10.7945,  -4.4544, -13.8432,  -9.6555,
         -23.1260, -17.5195,  -5.1675, -12.6809, -11.0318, -20.5066, -24.0512,
           4.7974,  -6.1301]])
tensor([[ 0.5661,  1.4662,  1.3178,  0.0593, -0.1055,  1.4247, -0.2955,  1.5350,
          1.1177,  1.2165,  1.8881,  1.8321,  1.2974,  1.6981,  1.4894,  0.3009,
          1.4307,  1.0161,  1.3361, -0.4675,  0.7388,  0.9840,  1.0629,  1.1601,
          1.4592,  0.4900,  1.1667,  1.1513,  1.1988,  1.4536]],
       grad_fn=<AddmmBackward0>)
i: 115
Mesh labels: [28]
tensor([[ -5.5571, -16.3994, -14.1083, -13.3570, -10.1724,  -9.5266,  -7.1264,
         -13.0232, -19.3412, -16.8826,  -8.5119,  -7.4851,  -6.3209, -18.7097,
          -4.8544, -12.0904,  -8.6519, -12.1154,  -5.3973, -13.2534, -10.3211,
         -24.7786, -18.1657,  -5.0330, -12.2443, -

In [57]:
# Record testing accuracy after attack
attacked_model_outputs = test_attacked_model(mesh_cnn, dataloader, opt_writer)

epoch: -1, TEST ACC: [99.167 %]

